# Postes km

In [199]:
import pandas as pd

postes_viejo = pd.read_excel("postes_km_anterior.xlsx")

nuevos_nombres = {
    "ruta": "Ruta",
    "km": "KM",
    "longitude": "Lat",
    "latitude": "Long"
}
postes_viejo.rename(columns=nuevos_nombres, inplace=True)

postes_viejo = postes_viejo[["Ruta","KM","Lat","Long"]]
postes_viejo

,Ruta,KM,Lat,Long
0,7,345,-54.585138,-32.575664
1,7,349,-54.570987,-32.542146
2,7,355,-54.525446,-32.513338
3,7,365,-54.441941,-32.468569
4,7,370,-54.406333,-32.435834
...,...,...,...,...
6103,200,48,-55.738036,-34.759697
6104,200,47,-55.747790,-34.763694
6105,200,50,-55.718451,-34.752856
6106,200,51,-55.707844,-34.750046


In [200]:
postes_relevados = pd.read_excel("radares_geolocalizacion.xlsx", sheet_name="Postes")
postes_relevados = postes_relevados[["Ruta","KM","Lat","Long"]]
postes_relevados

,Ruta,KM,Lat,Long
0,IB,21,-34.831003,-56.010360
1,IB,22,-34.826627,-56.000459
2,IB,24,-34.818999,-55.982176
3,IB,28,-34.798131,-55.946357
4,IB,32,-34.787967,-55.906642
...,...,...,...,...
217,1,4,-34.872734,-56.237084
218,1,3,-34.870019,-56.227237
219,1,2,-34.871604,-56.217586
220,1,100,-34.371451,-57.043962


In [201]:
postes_combinados = pd.concat([postes_viejo, postes_relevados], ignore_index=True)

postes_combinados['Ruta'] = postes_combinados['Ruta'].replace(200, 'IB')
postes_combinados_sin_duplicados = postes_combinados.drop_duplicates(subset=["Ruta", "KM"], keep="first")


postes_combinados_sin_duplicados

,Ruta,KM,Lat,Long
0,7,345,-54.585138,-32.575664
1,7,349,-54.570987,-32.542146
2,7,355,-54.525446,-32.513338
3,7,365,-54.441941,-32.468569
4,7,370,-54.406333,-32.435834
...,...,...,...,...
6319,1,52,-34.641230,-56.627981
6322,1,122,-34.334930,-57.270923
6324,1,2,-34.871735,-56.217147
6325,1,4,-34.872734,-56.237084


In [202]:
import pandas as pd

# Definir los rangos válidos para Uruguay
LAT_MIN, LAT_MAX = -35.5, -30.0
LONG_MIN, LONG_MAX = -58.5, -53.0

def corregir_coordenadas(row):
    lat, long = row['Lat'], row['Long']
    
    # Verificar si el punto está dentro del rango válido
    if LAT_MIN <= lat <= LAT_MAX and LONG_MIN <= long <= LONG_MAX:
        return lat, long  # Coordenadas correctas, no hacer nada
    
    # Rotar las coordenadas si están fuera del rango
    if LAT_MIN <= long <= LAT_MAX and LONG_MIN <= lat <= LONG_MAX:
        return long, lat  # Intercambiar latitud y longitud
    
    # Dejar como NaN si sigue fuera del rango después de rotar
    return pd.NA, pd.NA

# Aplicar la corrección al DataFrame
postes_combinados_sin_duplicados[['Lat', 'Long']] = postes_combinados_sin_duplicados.apply(
    lambda row: pd.Series(corregir_coordenadas(row)), axis=1
)

# Eliminar filas con coordenadas que siguen siendo inválidas (opcional)
postes_combinados_sin_duplicados = postes_combinados_sin_duplicados.dropna(subset=['Lat', 'Long'])

postes_combinados_sin_duplicados


C:\Users\fpino\AppData\Local\Temp\ipykernel_2264\185358471.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postes_combinados_sin_duplicados[['Lat', 'Long']] = postes_combinados_sin_duplicados.apply(


,Ruta,KM,Lat,Long
0,7,345,-32.575664,-54.585138
1,7,349,-32.542146,-54.570987
2,7,355,-32.513338,-54.525446
3,7,365,-32.468569,-54.441941
4,7,370,-32.435834,-54.406333
...,...,...,...,...
6319,1,52,-34.641230,-56.627981
6322,1,122,-34.334930,-57.270923
6324,1,2,-34.871735,-56.217147
6325,1,4,-34.872734,-56.237084


In [203]:
postes_combinados_sin_duplicados.to_excel("Postes_KM_2025.xlsx", index=False)

# Linea Base Carteles

In [104]:
carteles = pd.read_excel("radares_geolocalizacion.xlsx", sheet_name="Linea Base")
carteles

,Nombre de video,Lat,Long,Código Radar
0,GX010067,-34.709598,-56.207375,M108
1,GX010067,-34.704995,-56.202791,M108
2,GX010068,-34.601586,-56.261079,M023 A
3,GX010068,-34.593040,-56.262199,M023 B
4,GX010068,-34.591447,-56.262751,M023 B
...,...,...,...,...
244,GX010127,-33.512603,-56.913041,M056
245,GX010128,-33.515000,-56.883330,M058
246,GX010128,-33.515126,-56.884597,M058
247,GX010129,-33.536184,-56.888858,M057


In [105]:
df_agrupado = carteles.sort_values(by="Código Radar")

df_agrupado

,Nombre de video,Lat,Long,Código Radar
92,GX010024,-34.828071,-56.003778,M001 - A
91,GX010024,-34.829707,-56.007576,M001 - A
134,GX020025,-34.824717,-55.996601,M001 - B
93,GX010024,-34.804940,-55.956526,M002 - A
133,GX020025,-34.799423,-55.948591,M002 - B
...,...,...,...,...
127,GX020024,-34.766409,-55.761199,No corresponde
104,GX010025,-34.765050,-55.752932,No corresponde
95,GX010024,-34.785897,-55.889999,No corresponde
94,GX010024,-34.787301,-55.892055,No corresponde


In [106]:
df_agrupado["Código Radar"] = df_agrupado["Código Radar"].str.replace(" ", "", regex=False)
df_agrupado["Código Radar"] = df_agrupado["Código Radar"].str.upper()  

In [107]:
df_agrupado["Código Radar"].value_counts()

Código Radar
M010             12
NOCORRESPONDE     5
M018              5
M008              4
M058              4
                 ..
M064              1
M045              1
M046              1
M091              1
M092              1
Name: count, Length: 119, dtype: int64

In [108]:
df_agrupado[df_agrupado["Código Radar"]=="M010"]


,Nombre de video,Lat,Long,Código Radar
40,GX010089,-33.874620,-57.367800,M010
51,GX010091,-33.874775,-57.372222,M010
50,GX010091,-33.882646,-57.372031,M010
49,GX010091,-33.890082,-57.372166,M010
48,GX010091,-33.876946,-57.372316,M010
47,GX010091,-33.874927,-57.374667,M010
52,GX010091,-33.874497,-57.365243,M010
46,GX010091,-33.874970,-57.375892,M010
42,GX010089,-33.877623,-57.357376,M010
41,GX010089,-33.874505,-57.365159,M010


In [109]:
df_agrupado.to_excel("Carteles_ordenado.xlsx", index=False)

PermissionError: [Errno 13] Permission denied: 'Carteles_ordenado.xlsx'

comentario: la idea es usar una logica para quedarme con un sub data frame donde tengamos solo un sentido en algunos casos y doble snetido en el otro, por lo tanto deberiamos hacer este siguiente codigo y buscar los nombres qu nos gustaria o bien implementar logica donde se logre identigficar que tenemos a + y - buscando un prefijo comun y solo la variante de carcater finañl 

In [125]:
radares = pd.read_excel("radares_geolocalizacion.xlsx", sheet_name="Radares")
radares["Código Radar"] = radares["Código Radar"].str.replace(" ", "", regex=False)
radares["Código Radar"] = radares["Código Radar"].str.upper()  
radares.head(10)

,Nombre Video,Lat,Long,Código Radar
0,GX020024,-34.827427,-56.002304,M001-A
1,GX020025,-34.825792,-55.998950,M001-B
2,GX010024,-34.803354,-55.954161,M002-A
3,GX010024,-34.780609,-55.873260,M003-A
4,GX020025,-34.780124,-55.871923,M003-B
5,GX010024,-34.777975,-55.857361,M004-A
6,GX010025,-34.771885,-55.823007,M005-A
7,GX010024,-34.771719,-55.821438,M005-B
8,GX010079,-34.591356,-56.703672,M007
9,GX010082,-34.336311,-57.259966,M008


In [154]:
valores_a_filtrar = [
    "M101", "M102", "M103", "M104", "M105",
    "M107", "M108", "M109", "M110","M111",
    "M112", "M112", "M113", "M114","M115"
]


subcadenas = []
for index, row in df_agrupado.iterrows():
    subcadena_encontrada = None
    for valor in valores_a_filtrar:
        if valor in row["Código Radar"]:
            subcadena_encontrada = valor
            break
    subcadenas.append(subcadena_encontrada)

df_agrupado["Subcadena"] = subcadenas


carteles_filtrado = df_agrupado[df_agrupado["Subcadena"].notna()]

carteles_filtrado

,Nombre de video,Lat,Long,Código Radar,Subcadena
223,GX010118,-31.417612,-57.939999,M101,M101
222,GX010118,-31.417648,-57.939970,M101,M101
221,GX010118,-31.414117,-57.942704,M101,M101
30,GX010084,-34.444148,-57.837359,M102,M102
31,GX010084,-34.440489,-57.838153,M102,M102
32,GX010086,-34.034845,-57.901721,M103,M103
33,GX010086,-34.029713,-57.897523,M103,M103
34,GX010087,-33.979385,-58.292338,M104,M104
35,GX010087,-33.980959,-58.289127,M104,M104
10,GX010072,-34.424652,-56.279554,M105,M105


In [155]:
radares_filtrado = radares[radares["Código Radar"].isin(valores_a_filtrar)]

radares_filtrado

,Nombre Video,Lat,Long,Código Radar
81,GX010118,-31.415892,-57.941332,M101
82,GX010084,-34.445776,-57.837290,M102
83,GX010086,-34.032025,-57.899700,M103
84,GX010087,-33.981080,-58.288348,M104
85,GX010072,-34.425279,-56.282218,M105
86,GX010073,-34.644944,-56.068053,M107
87,GX010067,-34.704276,-56.201969,M108
88,GX010067,-34.707162,-56.204991,M108
89,GX010074,-34.643536,-56.070499,M109
90,GX010022,-34.754409,-56.019251,M110


In [149]:
radares[radares["Código Radar"].isin(["M004"])]

,Nombre Video,Lat,Long,Código Radar


In [156]:
import pandas as pd
import folium
import random
from geopy.distance import geodesic

# Crear el mapa centrado en las coordenadas promedio de `carteles_filtrado`
mapa = folium.Map(location=[carteles_filtrado["Lat"].mean(), carteles_filtrado["Long"].mean()], zoom_start=13)

# Generar colores más oscuros únicos para cada valor de "Código Radar"
valores_unicos = pd.concat([carteles_filtrado["Código Radar"], radares_filtrado["Código Radar"]]).unique()
colores = {valor: f"#{random.randint(0, 0x7F7F7F):06x}" for valor in valores_unicos}  # Colores más oscuros

# Función para encontrar el punto más cercano
def encontrar_mas_cercano(origen, puntos_disponibles):
    return min(
        puntos_disponibles,
        key=lambda destino: geodesic((origen["Lat"], origen["Long"]), (destino["Lat"], destino["Long"])).meters
    )

# Procesar cada grupo de "Código Radar"
for codigo_radar in valores_unicos:
    puntos_grupo = carteles_filtrado[carteles_filtrado["Subcadena"] == codigo_radar].copy()
    marcador_estandar = radares_filtrado[radares_filtrado["Código Radar"] == codigo_radar]
    
    if marcador_estandar.empty:
        continue

    marcador_final = {
        "Lat": marcador_estandar.iloc[0]["Lat"],
        "Long": marcador_estandar.iloc[0]["Long"]
    }
    
    # Generar las líneas conectando cada punto al más cercano
    puntos_disponibles = puntos_grupo.to_dict("records")
    if puntos_disponibles:
        punto_actual = puntos_disponibles.pop(0)  # Tomar el primer punto

        while puntos_disponibles:
            siguiente_punto = encontrar_mas_cercano(punto_actual, puntos_disponibles)
            folium.PolyLine(
                locations=[(punto_actual["Lat"], punto_actual["Long"]), (siguiente_punto["Lat"], siguiente_punto["Long"])],
                color=colores[codigo_radar],
                weight=3  # Grosor de la línea
            ).add_to(mapa)
            puntos_disponibles.remove(siguiente_punto)
            punto_actual = siguiente_punto
        
        # Conectar el marcador estándar al punto más cercano
        punto_cercano = encontrar_mas_cercano(marcador_final, puntos_grupo.to_dict("records"))
        folium.PolyLine(
            locations=[(punto_cercano["Lat"], punto_cercano["Long"]), (marcador_final["Lat"], marcador_final["Long"])],
            color=colores[codigo_radar],
            weight=3  # Grosor de la línea
        ).add_to(mapa)

    # Añadir CircleMarkers para el grupo
    for _, row in carteles_filtrado[carteles_filtrado["Subcadena"] == codigo_radar].iterrows():
        folium.CircleMarker(
            location=[row["Lat"], row["Long"]],
            radius=7,
            color=colores[codigo_radar],
            fill=True,
            fill_color=colores[codigo_radar],
            fill_opacity=0.99,
            popup=f"Código Radar: {row['Código Radar']}"
        ).add_to(mapa)

    # Añadir marcador estándar para el grupo
    folium.Marker(
        location=[marcador_final["Lat"], marcador_final["Long"]],
        popup=f"Radar estándar: {codigo_radar}",
        icon=folium.Icon(color="white", icon_color=colores[codigo_radar])
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save("mapa_geografico_9.html")
print("Mapa interactivo guardado como 'mapa_geografico.html'")


Mapa interactivo guardado como 'mapa_geografico.html'


# Muesta para desarrollo 

### MAPA CON RADARES SELECCIONADOS 

In [162]:
carteles = pd.read_excel("Radares_prueba.xlsx", sheet_name="Linea Base")
radares = pd.read_excel("Radares_prueba.xlsx", sheet_name="Radares")


valores_a_filtrar = [
    "M001-A", "M001-B", "M084-E", "M084-S", "M102", "M104", "M108"
]

subcadenas = []
for index, row in carteles.iterrows():
    subcadena_encontrada = None
    for valor in valores_a_filtrar:
        if valor in row["Código Radar"]:
            subcadena_encontrada = valor
            break
    subcadenas.append(subcadena_encontrada)

carteles["Subcadena"] = subcadenas
carteles_filtrado = carteles[carteles["Subcadena"].notna()]


radares_filtrado = radares[radares["Código Radar"].isin(valores_a_filtrar)]



# Crear el mapa centrado en las coordenadas promedio de `carteles_filtrado`
mapa = folium.Map(location=[carteles_filtrado["Lat"].mean(), carteles_filtrado["Long"].mean()], zoom_start=13)

# Generar colores más oscuros únicos para cada valor de "Código Radar"
valores_unicos = pd.concat([carteles_filtrado["Código Radar"], radares_filtrado["Código Radar"]]).unique()
colores = {valor: f"#{random.randint(0, 0x7F7F7F):06x}" for valor in valores_unicos}  # Colores más oscuros

# Función para encontrar el punto más cercano
def encontrar_mas_cercano(origen, puntos_disponibles):
    return min(
        puntos_disponibles,
        key=lambda destino: geodesic((origen["Lat"], origen["Long"]), (destino["Lat"], destino["Long"])).meters
    )

# Procesar cada grupo de "Código Radar"
for codigo_radar in valores_unicos:
    puntos_grupo = carteles_filtrado[carteles_filtrado["Subcadena"] == codigo_radar].copy()
    marcador_estandar = radares_filtrado[radares_filtrado["Código Radar"] == codigo_radar]
    
    if marcador_estandar.empty:
        continue

    marcador_final = {
        "Lat": marcador_estandar.iloc[0]["Lat"],
        "Long": marcador_estandar.iloc[0]["Long"]
    }
    
    # Generar las líneas conectando cada punto al más cercano
    puntos_disponibles = puntos_grupo.to_dict("records")
    if puntos_disponibles:
        punto_actual = puntos_disponibles.pop(0)  # Tomar el primer punto

        while puntos_disponibles:
            siguiente_punto = encontrar_mas_cercano(punto_actual, puntos_disponibles)
            folium.PolyLine(
                locations=[(punto_actual["Lat"], punto_actual["Long"]), (siguiente_punto["Lat"], siguiente_punto["Long"])],
                color=colores[codigo_radar],
                weight=3  # Grosor de la línea
            ).add_to(mapa)
            puntos_disponibles.remove(siguiente_punto)
            punto_actual = siguiente_punto
        
        # Conectar el marcador estándar al punto más cercano
        punto_cercano = encontrar_mas_cercano(marcador_final, puntos_grupo.to_dict("records"))
        folium.PolyLine(
            locations=[(punto_cercano["Lat"], punto_cercano["Long"]), (marcador_final["Lat"], marcador_final["Long"])],
            color=colores[codigo_radar],
            weight=3  # Grosor de la línea
        ).add_to(mapa)

    # Añadir CircleMarkers para el grupo
    for _, row in carteles_filtrado[carteles_filtrado["Subcadena"] == codigo_radar].iterrows():
        folium.CircleMarker(
            location=[row["Lat"], row["Long"]],
            radius=7,
            color=colores[codigo_radar],
            fill=True,
            fill_color=colores[codigo_radar],
            fill_opacity=0.99,
            popup=f"Código Radar: {row['Código Radar']}"
        ).add_to(mapa)

    # Añadir marcador estándar para el grupo
    folium.Marker(
        location=[marcador_final["Lat"], marcador_final["Long"]],
        popup=f"Radar estándar: {codigo_radar}",
        icon=folium.Icon(color="white", icon_color=colores[codigo_radar])
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save("mapa_geografico_sub-muestra.html")
print("Mapa interactivo guardado como 'mapa_geografico.html'")

Mapa interactivo guardado como 'mapa_geografico.html'


### Generación de carpeta con clips y frame 
 

In [197]:
import pandas as pd

carteles = pd.read_excel("Radares_prueba.xlsx", sheet_name="Linea Base")
radares = pd.read_excel("Radares_prueba.xlsx", sheet_name="Radares")

df = pd.read_csv('./dev/GX010022_HERO11 Black-GPS9 (1).csv')
df

,cts,date,GPS (Lat.) [deg],GPS (Long.) [deg],GPS (Alt.) [m],GPS (2D) [m/s],GPS (3D) [m/s],GPS (days) [deg],GPS (secs) [s],GPS (DOP) [deg],GPS (fix) [deg],altitude system
0,1.307800e+02,2024-08-08T17:16:03.199Z,-34.726454,-55.963808,28.717,2.914,2.92,8986,62163.199,1.22,3,MSLV
1,2.309829e+02,2024-08-08T17:16:03.298Z,-34.726455,-55.963810,28.719,2.705,2.92,8986,62163.299,1.22,3,MSLV
2,3.311858e+02,2024-08-08T17:16:03.398Z,-34.726456,-55.963813,28.675,2.851,2.71,8986,62163.399,1.22,3,MSLV
3,4.313887e+02,2024-08-08T17:16:03.499Z,-34.726458,-55.963816,28.649,2.917,2.85,8986,62163.499,1.22,3,MSLV
4,5.315916e+02,2024-08-08T17:16:03.599Z,-34.726459,-55.963818,28.647,2.788,2.92,8986,62163.599,1.22,3,MSLV
...,...,...,...,...,...,...,...,...,...,...,...,...
21136,2.113656e+06,2024-08-08T17:51:16.800Z,-34.793172,-56.116254,42.401,13.822,13.86,8986,64276.800,1.07,3,MSLV
21137,2.113747e+06,2024-08-08T17:51:16.900Z,-34.793179,-56.116267,42.417,13.899,13.82,8986,64276.900,1.07,3,MSLV
21138,2.113838e+06,2024-08-08T17:51:17.000Z,-34.793185,-56.116280,42.420,13.827,13.90,8986,64277.000,1.07,3,MSLV
21139,2.113929e+06,2024-08-08T17:51:17.099Z,-34.793192,-56.116293,42.429,13.825,13.83,8986,64277.100,1.07,3,MSLV


Lo que buscamos es determinar las rutas recorridas en un trayecto usando un enfoque basado en cercanía a los postes del DataFrame postes_km y luego limpiar los "falsos positivos" evaluando las secuencias de las rutas detectadas.

In [204]:
postes = pd.read_excel("Postes_KM_2025.xlsx")
postes

,Ruta,KM,Lat,Long
0,0,1,-34.873587,-56.206818
1,0,2,-34.871533,-56.217389
2,0,3,-34.869865,-56.227399
3,0,4,-34.872907,-56.237265
4,0,5,-34.867580,-56.245348
...,...,...,...,...
5695,IB,49,-34.756372,-55.727950
5696,IB,26,-34.809003,-55.962970
5697,IB,34,-34.783715,-55.886673
5698,IB,29,-34.791994,-55.937892


Fase 1: Determinar postes cercanos
Evaluar distancia a los postes (postes_km) para cada punto del trayecto (df):

Para cada fila en el DataFrame df, calcular cuáles postes están a menos de 25 metros.
Si hay varios postes dentro de los 25 metros, seleccionar el más cercano.
Guardar esta información en un nuevo DataFrame o lista con:
Índice del punto en df.
Ruta asociada al poste más cercano.
Distancia al poste más cercano.
Crear un DataFrame inicial ordenado:

El resultado de esta fase es un DataFrame que contiene una secuencia de rutas detectadas en el trayecto, con información sobre cuál poste (y ruta) se detectó como el más cercano a cada punto en df.

In [156]:
"""
trayectory_data = []

for index, row in df.iloc[::20].iterrows():
    punto_coords = (row['GPS (Lat.) [deg]'], row['GPS (Long.) [deg]'])
    
    distancia_minima = float('inf') # se inicializa en infito 
    ruta_cercana = None
    km_cercano = None
    
    for _, poste in postes.iterrows():

        poste_coords = (poste['Lat'], poste['Long'])

        distancia = geodesic(punto_coords, poste_coords).meters
        
        if distancia < 35 and distancia < distancia_minima:
            distancia_minima = distancia
            ruta_cercana = poste['Ruta']
            km_cercano = poste['KM']
    
    # ruta_cercana = None == FLASE --- ruta_cercana = a algo no falsy es true
    if ruta_cercana:
        trayectory_data.append({
            'GPS (Lat.) [deg]': row['GPS (Lat.) [deg]'],
            'GPS (Long.) [deg]': row['GPS (Long.) [deg]'],
            'Ruta': ruta_cercana,
            'KM': km_cercano,
            'Distancia': distancia_minima
        })

trayectory = pd.DataFrame(trayectory_data)

trayectory
"""

,GPS (Lat.) [deg],GPS (Long.) [deg],Ruta,KM,Distancia
0,-34.781110,-55.875958,IB,35,20.643092
1,-34.781179,-55.876243,IB,35,28.039465
2,-34.783729,-55.886476,IB,34,18.138325
3,-34.783808,-55.886685,IB,34,10.340156
4,-34.783887,-55.886883,IB,34,27.017482
5,-34.788559,-55.895811,IB,33,29.233361
6,-34.787935,-55.906091,IB,32,28.540076
7,-34.787855,-55.906405,IB,32,18.136114
8,-34.787710,-55.916898,IB,31,29.379363
9,-34.787776,-55.917215,IB,31,9.064150


In [205]:
import pandas as pd
from geopy.distance import geodesic

trayectory_data = []
found_posts = 0  # Contador de postes encontrados
last_position_index = 0  # Índice de la última posición procesada
accumulated_distance = 0  # Distancia acumulada desde el último poste

# Iterar sobre el DataFrame df
while last_position_index < len(df):
    row = df.iloc[last_position_index]
    punto_coords = (row['GPS (Lat.) [deg]'], row['GPS (Long.) [deg]'])

    distancia_minima = float('inf')  # Se inicializa en infinito
    ruta_cercana = None
    km_cercano = None

    # Buscar el poste más cercano
    for _, poste in postes.iterrows():
        poste_coords = (poste['Lat'], poste['Long'])
        distancia = geodesic(punto_coords, poste_coords).meters

        if distancia < 20 and distancia < distancia_minima:
            distancia_minima = distancia
            ruta_cercana = poste['Ruta']
            km_cercano = poste['KM']

    # Si se encuentra un poste cercano
    if ruta_cercana:
        trayectory_data.append({
            'GPS (Lat.) [deg]': row['GPS (Lat.) [deg]'],
            'GPS (Long.) [deg]': row['GPS (Long.) [deg]'],
            'Ruta': ruta_cercana,
            'KM': km_cercano,
            'Distancia': distancia_minima
        })

        found_posts += 1
        accumulated_distance = 0  # Reiniciar la distancia acumulada

        # Si ya se encontraron los primeros dos postes, activar el salto de 800 metros
        if found_posts >= 1:
            # Calcular la distancia acumulada hasta que se alcance 800 metros
            while accumulated_distance < 920 and last_position_index < len(df) - 1:
                current_row = df.iloc[last_position_index]
                next_row = df.iloc[last_position_index + 1]

                # Coordenadas actuales y siguientes
                current_coords = (current_row['GPS (Lat.) [deg]'], current_row['GPS (Long.) [deg]'])
                next_coords = (next_row['GPS (Lat.) [deg]'], next_row['GPS (Long.) [deg]'])

                # Calcular la distancia entre puntos
                distance = geodesic(current_coords, next_coords).meters

                # Calcular el tiempo transcurrido entre puntos
                time_diff = (next_row['cts'] - current_row['cts']) / 1000  # En segundos

                # Calcular la velocidad promedio (2D) entre puntos
                velocity = (current_row['GPS (2D) [m/s]'] + next_row['GPS (2D) [m/s]']) / 2

                # Incrementar la distancia acumulada
                accumulated_distance += distance
                last_position_index += 1

                # Si alcanzamos los 800 metros, romper el bucle interno
                if accumulated_distance >= 920:
                    break

    # Si no se encuentra un poste cercano, avanzar al siguiente índice
    else:
        last_position_index += 15  # Seguir buscando cada 10 filas inicialmente

trayectory = pd.DataFrame(trayectory_data)
trayectory


,GPS (Lat.) [deg],GPS (Long.) [deg],Ruta,KM,Distancia
0,-34.734508,-55.975449,8,29,18.280563
1,-34.740890,-55.983924,8,28,15.679281
2,-34.752248,-56.001085,8,26,18.808811
3,-34.757662,-56.009223,8,25,13.152962
4,-34.755451,-56.019033,74,25,8.074502
5,-34.747069,-56.022721,74,26,11.637666
6,-34.739258,-56.028142,74,27,6.427950
7,-34.746872,-56.022869,74,26,17.589936
8,-34.755388,-56.019022,74,25,5.128592
9,-34.763492,-56.017742,8,24,13.724376


In [189]:
"""
import folium
from folium import PolyLine

# Crear un mapa centrado en las coordenadas promedio del DataFrame
def plot_trajectory(df):
    center_lat = df['GPS (Lat.) [deg]'].mean()
    center_lon = df['GPS (Long.) [deg]'].mean()

    # Crear el mapa con un zoom inicial
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Agregar la trayectoria al mapa
    coordinates = list(zip(df['GPS (Lat.) [deg]'], df['GPS (Long.) [deg]']))
    PolyLine(coordinates, color="blue", weight=2.5, opacity=1).add_to(m)

    return m

# Llamar a la función para plotear la trayectoria completa
mapa_trayectoria = plot_trajectory(df)
mapa_trayectoria.save("trayectoria_completa.html")
"""

In [206]:
filtered_rows = []

i = 0
while i < len(trayectory):
    current_group = [trayectory.iloc[i]]
    
    # Comparar la fila actual con las siguientes
    while (
        i + 1 < len(trayectory) and
        trayectory.iloc[i]['Ruta'] == trayectory.iloc[i + 1]['Ruta'] and
        trayectory.iloc[i]['KM'] == trayectory.iloc[i + 1]['KM']
    ):
        current_group.append(trayectory.iloc[i + 1])
        i += 1
    
    # Quedarse con la fila de menor distancia dentro del grupo
    min_distance_row = min(current_group, key=lambda x: x['Distancia'])
    filtered_rows.append(min_distance_row)
    
    i += 1

trayectory_cleaned = pd.DataFrame(filtered_rows)



trayectory_cleaned


,GPS (Lat.) [deg],GPS (Long.) [deg],Ruta,KM,Distancia
0,-34.734508,-55.975449,8.0,29.0,18.280563
1,-34.740890,-55.983924,8.0,28.0,15.679281
2,-34.752248,-56.001085,8.0,26.0,18.808811
3,-34.757662,-56.009223,8.0,25.0,13.152962
4,-34.755451,-56.019033,74.0,25.0,8.074502
5,-34.747069,-56.022721,74.0,26.0,11.637666
6,-34.739258,-56.028142,74.0,27.0,6.427950
7,-34.746872,-56.022869,74.0,26.0,17.589936
8,-34.755388,-56.019022,74.0,25.0,5.128592
9,-34.763492,-56.017742,8.0,24.0,13.724376


Fase 2: Evaluar la secuencia de rutas
Una vez tenemos una secuencia inicial de rutas basada en la cercanía a los postes, implementamos una lógica de limpieza para identificar y descartar "falsos positivos" (postes de rutas perpendiculares o irrelevantes).

Identificar falsos positivos dentro de la secuencia:

Recorremos la secuencia de rutas y evaluamos patrones. Por ejemplo:
Si estamos en la ruta X y detectamos un cambio repentino a la ruta Y, seguido nuevamente por X (X, X, Y, X, X), descartamos Y.
Si encontramos rutas diferentes (Y, Z, E) que aparecen de forma aislada entre segmentos largos de una misma ruta (X, X, X, Y, Z, E, X, X), descartamos esas rutas.
La idea es que un cambio de ruta solo tiene sentido si se mantiene la nueva ruta por un segmento consecutivo de puntos.
Mantener cambios de ruta legítimos:

Si una nueva ruta aparece al principio o al final de la secuencia y se mantiene durante varios puntos consecutivos, consideramos que hubo un cambio de ruta válido.
También es válido un cambio de ruta si aparece un segmento claro y prolongado de una nueva ruta (por ejemplo, X, X, X, Y, Y, Y, Y, Z, Z, Z).



In [207]:
filtered_rows = []  # Lista para almacenar las filas filtradas
temp_rows = []      # Lista temporal para almacenar filas de una posible nueva ruta
current_route = None  # Ruta actual que se considera válida

# Umbral para considerar un cambio de ruta como válido
MIN_CONSECUTIVE_ROWS = 2

for index, row in trayectory_cleaned.iterrows():
    ruta_actual = row['Ruta']
    km_actual = row['KM']
    
    if current_route is None:
        # Si es la primera fila, inicializar la ruta actual y agregar la fila a `filtered_rows`
        current_route = ruta_actual
        filtered_rows.append(row)
        continue
    
    if ruta_actual == current_route:
        # Si la ruta es igual a la ruta válida actual, agregar la fila a `filtered_rows`
        filtered_rows.append(row)
        # Si hay filas temporales, descartarlas porque volvimos a la ruta válida
        if temp_rows:
            temp_rows = []
    else:
        # Si la ruta cambia, guardar la fila en `temp_rows`
        temp_rows.append(row)
        
        # Evaluar si hay suficientes filas consecutivas en la nueva ruta
        if len(temp_rows) >= MIN_CONSECUTIVE_ROWS:
            # Considerar el cambio de ruta como válido
            current_route = ruta_actual
            filtered_rows.extend(temp_rows)
            temp_rows = []

# Crear un nuevo DataFrame a partir de las filas filtradas
trayectory_final = pd.DataFrame(filtered_rows)

# Mostrar el DataFrame limpio
trayectory_final

,GPS (Lat.) [deg],GPS (Long.) [deg],Ruta,KM,Distancia
0,-34.734508,-55.975449,8.0,29.0,18.280563
1,-34.740890,-55.983924,8.0,28.0,15.679281
2,-34.752248,-56.001085,8.0,26.0,18.808811
3,-34.757662,-56.009223,8.0,25.0,13.152962
4,-34.755451,-56.019033,74.0,25.0,8.074502
5,-34.747069,-56.022721,74.0,26.0,11.637666
6,-34.739258,-56.028142,74.0,27.0,6.427950
7,-34.746872,-56.022869,74.0,26.0,17.589936
8,-34.755388,-56.019022,74.0,25.0,5.128592
9,-34.763492,-56.017742,8.0,24.0,13.724376


In [208]:
radares

,Nombre de video,Lat,Long,Código Radar
0,GX020024,-34.827427,-56.002304,M001-A
1,GX020025,-34.825792,-55.998950,M001-B
2,GX010014,-34.792346,-56.098803,M084-S
3,GX010022,-34.792648,-56.097112,M084-E
4,GX010084,-34.445776,-57.837290,M102
5,GX010087,-33.981080,-58.288348,M104
6,GX010067,-34.704276,-56.201969,M108


In [209]:
carteles

,Nombre de video,Lat,Long,Código Radar
0,GX010024,-34.829707,-56.007576,M001-A
1,GX010024,-34.828071,-56.003778,M001-A
2,GX020025,-34.824717,-55.996601,M001-B
3,GX010014,-34.791724,-56.101697,M084-E
4,GX010022,-34.792831,-56.080291,M084-S
5,GX010022,-34.793313,-56.093784,M084-S
6,GX010084,-34.444148,-57.837359,M102
7,GX010084,-34.440489,-57.838153,M102
8,GX010087,-33.979385,-58.292338,M104
9,GX010087,-33.980959,-58.289127,M104


In [210]:
from geopy.distance import geodesic

radares_encontrados = {}

for _, row_r in radares.iterrows():
    radar_coords = (row_r['Lat'], row_r['Long'])
    radar_codigo = row_r['Código Radar']  
    
    for start_d in range(0, len(df), 10):
        # Seleccionar la fila actual de `df`
        row_d = df.iloc[start_d]
        
        # Coordenadas del punto en `df`
        df_coords = (row_d['GPS (Lat.) [deg]'], row_d['GPS (Long.) [deg]'])
        
        # Calcular la distancia entre los puntos
        distance = geodesic(radar_coords, df_coords).meters
        
        if distance < 20:
            radares_encontrados[radar_codigo] = start_d
            break  # No necesitamos seguir buscando para este radar

radares_encontrados


{'M084-S': 19100, 'M084-E': 18980}

Filtra un DataFrame llamado carteles:
Usa un diccionario llamado radares_encontrados para obtener los nombres de radares y filtra el DataFrame carteles para que solo contenga las filas cuyos valores en la columna Código Radar coincidan con los nombres de los radares en el diccionario.

Inicializa columnas vacías:
Agrega dos columnas nuevas al DataFrame filtrado:

índice: Para guardar el índice del punto más cercano en el DataFrame df.
distancia: Para guardar la distancia mínima (en metros) entre el punto de carteles y el punto más cercano de df.
Encuentra el punto más cercano para cada fila:

Recorre cada fila de carteles_filtrado y calcula la distancia entre sus coordenadas (Lat, Long) y las coordenadas de todas las filas del DataFrame df (GPS (Lat.) [deg], GPS (Long.) [deg]).
Determina el punto más cercano en df a cada fila de carteles_filtrado y guarda:
El índice de ese punto en la columna índice.
La distancia mínima en la columna distancia.
Ordena el DataFrame resultante:
Una vez calculados los valores, ordena el DataFrame carteles_filtrado de menor a mayor según la columna índice.

Resultado final:
Devuelve un DataFrame carteles_filtrado que incluye las columnas originales de carteles junto con las columnas índice (índice del punto más cercano en df) y distancia (distancia al punto más cercano). Además, el DataFrame está ordenado por el índice del punto más cercano.

In [211]:
import pandas as pd
from geopy.distance import geodesic

# Filtrar el DataFrame `carteles` usando los nombres de los radares del diccionario `radares_encontrados`
nombres_radares = list(radares_encontrados.keys())
carteles_filtrado = carteles[carteles['Código Radar'].isin(nombres_radares)].copy()

# Inicializar nuevas columnas en el DataFrame filtrado
carteles_filtrado['índice'] = None
carteles_filtrado['distancia'] = None

# Recorrer cada fila del DataFrame filtrado
for index_c, row_c in carteles_filtrado.iterrows():
    # Coordenadas del punto en `carteles`
    carteles_coords = (row_c['Lat'], row_c['Long'])
    
    # Variables para almacenar la distancia mínima y el índice correspondiente
    distancia_minima = float('inf')
    indice_cercano = None
    
    # Recorrer todas las filas del DataFrame `df` para encontrar el punto más cercano
    for index_d, row_d in df.iterrows():
        # Coordenadas del punto en `df`
        df_coords = (row_d['GPS (Lat.) [deg]'], row_d['GPS (Long.) [deg]'])
        
        # Calcular la distancia entre los puntos
        distancia = geodesic(carteles_coords, df_coords).meters
        
        # Actualizar la distancia mínima y el índice si encontramos un punto más cercano
        if distancia < distancia_minima:
            distancia_minima = distancia
            indice_cercano = index_d
    
    # Guardar los valores calculados en las nuevas columnas
    carteles_filtrado.at[index_c, 'índice'] = indice_cercano
    carteles_filtrado.at[index_c, 'distancia'] = distancia_minima

carteles_filtrado = carteles_filtrado.sort_values(by='índice', ascending=True)




In [212]:
carteles_filtrado

,Nombre de video,Lat,Long,Código Radar,índice,distancia
4,GX010022,-34.792831,-56.080291,M084-S,17958,0.0
5,GX010022,-34.793313,-56.093784,M084-S,18768,0.0
3,GX010014,-34.791724,-56.101697,M084-E,19301,9.023882


In [213]:
trayectory_final

,GPS (Lat.) [deg],GPS (Long.) [deg],Ruta,KM,Distancia
0,-34.734508,-55.975449,8.0,29.0,18.280563
1,-34.740890,-55.983924,8.0,28.0,15.679281
2,-34.752248,-56.001085,8.0,26.0,18.808811
3,-34.757662,-56.009223,8.0,25.0,13.152962
4,-34.755451,-56.019033,74.0,25.0,8.074502
5,-34.747069,-56.022721,74.0,26.0,11.637666
6,-34.739258,-56.028142,74.0,27.0,6.427950
7,-34.746872,-56.022869,74.0,26.0,17.589936
8,-34.755388,-56.019022,74.0,25.0,5.128592
9,-34.763492,-56.017742,8.0,24.0,13.724376


In [214]:
from geopy.distance import geodesic

# Crear o inicializar la columna 'sentido' en el DataFrame `carteles_filtrados`
carteles_filtrado['sentido'] = None

# Recorrer cada fila de `carteles_filtrados`
for index_c, row_c in carteles_filtrado.iterrows():
    # Obtener el punto geográfico actual
    punto_cartel = (row_c['Lat'], row_c['Long'])
    
    # Variable para almacenar la suma mínima y los índices de los postes que la generaron
    suma_minima = float('inf')
    poste_inicial = None
    poste_final = None
    
    # Iterar sobre pares consecutivos de puntos en `trayectory_final`
    for i in range(len(trayectory_final) - 1):
        # Obtener las coordenadas de los postes consecutivos
        punto_inicial = (trayectory_final.iloc[i]['GPS (Lat.) [deg]'], trayectory_final.iloc[i]['GPS (Long.) [deg]'])
        punto_final = (trayectory_final.iloc[i + 1]['GPS (Lat.) [deg]'], trayectory_final.iloc[i + 1]['GPS (Long.) [deg]'])
        
        # Calcular las distancias desde el punto actual de `carteles_filtrados`
        distancia_inicial = geodesic(punto_cartel, punto_inicial).meters
        distancia_final = geodesic(punto_cartel, punto_final).meters
        
        # Calcular la suma de las distancias
        suma_distancias = distancia_inicial + distancia_final
        
        # Si la suma actual es menor que la suma mínima, actualizar
        if suma_distancias < suma_minima:
            suma_minima = suma_distancias
            poste_inicial = i
            poste_final = i + 1
    
    # Identificar el sentido según los valores de KM
    km_inicial = trayectory_final.iloc[poste_inicial]['KM']
    km_final = trayectory_final.iloc[poste_final]['KM']
    
    if km_inicial < km_final:
        carteles_filtrado.at[index_c, 'sentido'] = '+'
    else:
        carteles_filtrado.at[index_c, 'sentido'] = '-'

# Mostrar el DataFrame `carteles_filtrados` actualizado
carteles_filtrado


,Nombre de video,Lat,Long,Código Radar,índice,distancia,sentido
4,GX010022,-34.792831,-56.080291,M084-S,17958,0.0,+
5,GX010022,-34.793313,-56.093784,M084-S,18768,0.0,+
3,GX010014,-34.791724,-56.101697,M084-E,19301,9.023882,+


In [217]:
# Filtrar el DataFrame para mantener solo las filas que no cumplen la condición
carteles_filtrado = carteles_filtrado[
    ~((carteles_filtrado['sentido'] == '+') & (carteles_filtrado['Código Radar'].str.contains('-E', na=False)))
]

# Mostrar el DataFrame resultante
carteles_filtrado

,Nombre de video,Lat,Long,Código Radar,índice,distancia,sentido
4,GX010022,-34.792831,-56.080291,M084-S,17958,0.0,+
5,GX010022,-34.793313,-56.093784,M084-S,18768,0.0,+


In [ ]:
import os
import cv2
import pandas as pd

# Ruta base donde se encuentran los videos (mismo directorio que el script, en la carpeta "dev")
base_video_path = os.path.join(os.getcwd(), "dev")

# Ruta base para las carpetas que se crearán
output_base_path = os.path.join(base_video_path, "test-10022")

# Diccionario para contar las subcarpetas por "Código Radar"
carpetas_creadas = {}

# Recorrer cada fila de carteles_filtrados
for index, row in carteles_filtrado.iterrows():
    # Obtener el nombre de la carpeta principal por "Código Radar"
    folder_name = row['Código Radar']
    folder_path = os.path.join(output_base_path, folder_name)

    # Inicializar el contador de subcarpetas si es la primera vez que encontramos este "Código Radar"
    if folder_name not in carpetas_creadas:
        carpetas_creadas[folder_name] = 0

    # Incrementar el contador y definir el nombre de la subcarpeta
    carpetas_creadas[folder_name] += 1
    subfolder_name = f"cartel {carpetas_creadas[folder_name]}"

    # Crear las subcarpetas para frames y clips
    subfolder_frames_path = os.path.join(folder_path, "frames", subfolder_name)
    subfolder_clips_path = os.path.join(folder_path, "clips", subfolder_name)
    os.makedirs(subfolder_frames_path, exist_ok=True)
    os.makedirs(subfolder_clips_path, exist_ok=True)

    # Obtener el índice y buscar el valor 'cts' en el DataFrame 'df'
    indice = row['índice']
    cts_value = df.loc[indice, 'cts']  # Acceder al valor en el DataFrame df
    center_msec = cts_value  # Mantener los milisegundos directamente

    # Ubicar el video correspondiente (agregar extensión .mp4 si no está incluida)
    video_name = f"{row['Nombre de video']}.mp4"
    video_path = os.path.join(base_video_path, video_name)

    # Verificar si el video existe
    if not os.path.exists(video_path):
        print(f"El video {video_name} no existe en {base_video_path}. Saltando fila...")
        continue

    # Procesar el video para generar el clip
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"No se pudo abrir el video {video_name}. Saltando fila...")
        continue

    # Calcular el tiempo de inicio del clip (en milisegundos)
    clip_start_msec = max(0, center_msec - 5000)  # 5 segundos antes del centro
    clip_end_msec = center_msec + 5000  # 5 segundos después del centro

    # Configurar el códec para el clip
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)  # FPS promedio (por compatibilidad)
    clip_path = os.path.join(subfolder_clips_path, f"{folder_name}_clip.mp4")
    out = cv2.VideoWriter(clip_path, fourcc, fps, (frame_width, frame_height))

    # Posicionarse en el tiempo de inicio del clip
    cap.set(cv2.CAP_PROP_POS_MSEC, clip_start_msec)

    # Leer y escribir frames hasta el tiempo de finalización
    while cap.get(cv2.CAP_PROP_POS_MSEC) <= clip_end_msec:
        ret, frame = cap.read()
        if not ret:
            break

        # Agregar texto al frame
        minutes = int(center_msec // 60000)
        seconds = int((center_msec % 60000) // 1000)
        text = f"{video_name}, Start: {minutes:02d}:{seconds:02d}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        font_color = (0, 0, 255)  # Rojo
        thickness = 2
        text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_x = frame_width - text_size[0] - 15
        text_y = 40
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, font_color, thickness)

        out.write(frame)

    # Liberar recursos del clip
    cap.release()
    out.release()

    # Procesar el clip generado para extraer frames
    clip_cap = cv2.VideoCapture(clip_path)
    if not clip_cap.isOpened():
        print(f"No se pudo abrir el clip generado {clip_path}. Saltando...")
        continue

    # Calcular los tiempos para los 10 frames equiespaciados entre 2.5 y 7.5 segundos
    clip_fps = clip_cap.get(cv2.CAP_PROP_FPS)
    start_frame = int(2.5 * clip_fps)
    end_frame = int(7.5 * clip_fps)
    step = (end_frame - start_frame) // 10
    frames_to_extract = [start_frame + i * step for i in range(10)]

    # Extraer los frames
    for frame_number in frames_to_extract:
        clip_cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = clip_cap.read()
        if ret:
            # Agregar texto al frame
            minutes = int(center_msec // 60000)
            seconds = int((center_msec % 60000) // 1000)
            text = f"{video_name}, Start: {minutes:02d}:{seconds:02d}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.5
            font_color = (0, 0, 255)  # Rojo
            thickness = 2
            text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
            text_x = frame_width - text_size[0] - 15
            text_y = 40
            cv2.putText(frame, text, (text_x, text_y), font, font_scale, font_color, thickness)

            # Guardar el frame
            frame_name = f"frame_{frame_number}.jpg"
            frame_path = os.path.join(subfolder_frames_path, frame_name)
            cv2.imwrite(frame_path, frame)

    # Liberar recursos del clip
    clip_cap.release()

print("Proceso completado.")
